In [1]:
import time
import urllib.request, json 
import numpy as np
import json
import zmq
import binascii
import struct
import math
import base64
import threading
import bokeh
import csv

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models.widgets import PreText
from bokeh.resources import INLINE
from ipywidgets import interact

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
uuid=[]
charac=[]
listCharac=[]
cache={}
globaluuid=""
globalcharac=""
ct=0
initialTime =0
endTime =0
dataList=[]
flag=0

In [4]:
def updateCache():
    requests = zmq.Context().socket(zmq.REQ)
    requests.connect('tcp://137.132.165.139:5569')
    global cache
    requests.send_string("CACHE_REQUEST")
    cache = requests.recv_json()
    if cache and 'version' in cache:
        print('loaded cache version', cache['version'])
updateCache()

for gatt in cache['gatt']:
#     cha=[]
    uuid.append(gatt)
#     cha.append(cache['gatt'][gatt]['characteristics'][2:-2])
#     listCharac.append(cha)
    charac.append(cache['gatt'][gatt]['characteristics'][2:-2])
# print(charac)
# print(listCharac)

loaded cache version 1330


In [5]:
def endProcess(name):
    global dataList, initialTime, endTime, flag
    print("Process Ended")
    totalPack=(endTime-initialTime)/20
    receivePack = len(dataList)
    efficiency = receivePack / totalPack * 100
    print("Efficiency: " + str(efficiency))
    write_file = name
    with open(write_file, "w") as output:
        for line in dataList:
            output.write(line + '\n')
    initialTime=0
    endTime=0
    flag+=1
    print("Saved to file")

In [6]:
def homeRehab(x):
    global initialTime, endTime, dataList
#     x = requests.recv_json()
    d = base64.b64decode(x['data']) #x['data'].decode('base64')
    data=bytearray(d)
    ts0 = data[2] & 0x0f;
    ts1 = data[4] & 0x0f;
    ts2 = data[6] & 0x0f;
    ts3 = (data[8] & 0xf0) >> 4;
    ts4 = (data[10] & 0xf0) >> 4;
    ts5 = (data[12] & 0xf0) >> 4;
    timestamp = ts0 | (ts1<<4) | (ts2 <<8) | (ts3 << 12) | (ts4 << 16) | (ts5 << 20);
    
    ax = (data[2] & 0xf0) | (data[3] << 8);
    ay = (data[4] & 0xf0) | (data[5] << 8);
    az = (data[6] & 0xf0) | (data[7] << 8);
    accX =(ax + 2**15) % 2**16 - 2**15
    accY =(ay + 2**15) % 2**16 - 2**15
    accZ =(az + 2**15) % 2**16 - 2**15
    
    mx = ((data[8] & 0x0f)<<8 | data[9])<<4
    my = ((data[10] & 0x0f)<<8 | data[11])<<4
    mz = ((data[12] & 0x0f)<<8 | data[13])<<4
    magX =(mx + 2**15) % 2**16 - 2**15
    magY =(my + 2**15) % 2**16 - 2**15
    magZ =(mz + 2**15) % 2**16 - 2**15
    
    gx = data[14]<<8 | data[15]
    gy = data[16]<<8 | data[17]
    gz = data[18]<<8 | data[19]
    gyroX =(gx + 2**15) % 2**16 - 2**15
    gyroY =(gy + 2**15) % 2**16 - 2**15
    gyroZ =(gz + 2**15) % 2**16 - 2**15
    
    if flag==0:
        if initialTime ==0:
            dataList=[]
            initialTime=timestamp
            endTime = initialTime + (5*60*1000)
            dataList.append("accX,accY,accZ,magX,magY,magZ,gyroX,gyroY,gyroZ,timeStamp")
        else:
            if timestamp>endTime:
                endProcess("homeOutput.csv")
    
#     print(mx, my, mz, magX , magY , magZ)
    dataList.append(str(accX) + "," + str(accY) + "," + str(accZ) + "," + str(magX) + "," + str(magY) + "," + str(magZ) + "," + str(gyroX) + "," + str(gyroY) + "," + str(gyroZ) + "," + str(timestamp))
    new_dataAx = dict(x=[timestamp],y=[accX])
    new_dataAy = dict(x=[timestamp],y=[accY])
    new_dataAz = dict(x=[timestamp],y=[accZ])
    
    new_dataMx = dict(x=[timestamp],y=[magX])
    new_dataMy = dict(x=[timestamp],y=[magY])
    new_dataMz = dict(x=[timestamp],y=[magZ])
    
    new_dataGx = dict(x=[timestamp],y=[gyroX])
    new_dataGy = dict(x=[timestamp],y=[gyroY])
    new_dataGz = dict(x=[timestamp],y=[gyroZ])
    
    sourceAx.stream(new_dataAx,100)
    sourceAy.stream(new_dataAy,100)
    sourceAz.stream(new_dataAz,100)
    
    sourceMx.stream(new_dataMx,100)
    sourceMy.stream(new_dataMy,100)
    sourceMz.stream(new_dataMz,100)
    
    sourceGx.stream(new_dataGx,100)
    sourceGy.stream(new_dataGy,100)
    sourceGz.stream(new_dataGz,100)
    
    push_notebook(handle=handle)
#     time.sleep(1)

In [7]:
def heartRate(x):
    global ct, flag, dataList, initialTime, endTime
    ct+=1
    
    d = base64.b64decode(x['data']) #x['data'].decode('base64')
    data=bytearray(d)
    
    HRMEasurement = (data[1] + 2**15) % 2**16 - 2**15
    
    if flag==0:
        if initialTime ==0:
            dataList=[]
            initialTime=int(time.time())
            endTime = initialTime + (5*60)
            dataList.append("hr,timeStamp")
        else:
            if int(time.time())>endTime:
                endProcess("hrOutput.csv")
    
#     print(mx, my, mz, magX , magY , magZ)
    dataList.append(str(HRMEasurement) + str(int(time.time())))
    
    
    new_dataAx = dict(x=[int(time.time())],y=[HRMEasurement])
    sourceAx.stream(new_dataAx,100)
    
    push_notebook(handle=handle)
    time.sleep(1)

In [8]:
def updateUUID(f):
    global globaluuid
    globaluuid=f

In [9]:
def updateCharac(f):
    global globalcharac
    globalcharac=f

In [10]:
sourceAx = ColumnDataSource(dict(x=[],y=[]))
sourceAy = ColumnDataSource(dict(x=[],y=[]))
sourceAz = ColumnDataSource(dict(x=[],y=[]))

sourceMx = ColumnDataSource(dict(x=[],y=[]))
sourceMy = ColumnDataSource(dict(x=[],y=[]))
sourceMz = ColumnDataSource(dict(x=[],y=[]))

sourceGx = ColumnDataSource(dict(x=[],y=[]))
sourceGy = ColumnDataSource(dict(x=[],y=[]))
sourceGz = ColumnDataSource(dict(x=[],y=[]))

fig=figure(plot_width=900, plot_height=400)
fig.line(source=sourceAx, x='x', y='y', line_width=2, alpha=.85, color='red')
fig.line(source=sourceAy, x='x', y='y', line_width=2, alpha=.65, color='red')
fig.line(source=sourceAz, x='x', y='y', line_width=2, alpha=.45, color='red')
fig.line(source=sourceMx, x='x', y='y', line_width=2, alpha=.85, color='blue')
fig.line(source=sourceMy, x='x', y='y', line_width=2, alpha=.65, color='blue')
fig.line(source=sourceMz, x='x', y='y', line_width=2, alpha=.45, color='blue')
fig.line(source=sourceGx, x='x', y='y', line_width=2, alpha=.85, color='green')
fig.line(source=sourceGy, x='x', y='y', line_width=2, alpha=.65, color='green')
fig.line(source=sourceGz, x='x', y='y', line_width=2, alpha=.45, color='green')

stats = PreText(text='', width=500)

# ticker1 = Select(value='b4994c579099', options=uuid, title="UUID")
# ticker2 = Select(value='fff1', options=charac, title="Characteristic")

handle = show(column(fig), notebook_handle=True)

requests = zmq.Context().socket(zmq.PULL)
requests.bind('tcp://137.132.165.161:5566')

global globaluuid, globalcharac
interact(updateUUID,f=uuid)
interact(updateCharac, f=charac)

def main():
    while True:
#         print(globaluuid + " : " + globalcharac)
#         time.sleep(1)
        x = requests.recv_json()
        d = x['uuid']
        e = x['characteristic']
        if d==globaluuid:
            if e==globalcharac:
                if e== "fff1":
                    homeRehab(x)
                elif e== "2a37":
                    heartRate(x)
                else:
                    print("Device not supported")
                    time.sleep(1)
            else:
                print("Characteristic not found")
                time.sleep(1)
                    
t1 = threading.Thread(target=main)
t1.setDaemon(True)
t1.start()

        
# figAcc=Figure(plot_width=900, plot_height=400, y_range=[-20000,20000])
# figAcc.line(source=sourceAx, x='x', y='y', line_width=2, alpha=.85, color='red')
# figAcc.line(source=sourceAy, x='x', y='y', line_width=2, alpha=.85, color='blue')
# figAcc.line(source=sourceAz, x='x', y='y', line_width=2, alpha=.85, color='green')

# figMag=Figure(plot_width=900, plot_height=400, y_range=[-20000,20000])
# figMag.line(source=sourceMx, x='x', y='y', line_width=2, alpha=.85, color='red')
# figMag.line(source=sourceMy, x='x', y='y', line_width=2, alpha=.85, color='blue')
# figMag.line(source=sourceMz, x='x', y='y', line_width=2, alpha=.85, color='green')

# figGyro=Figure(plot_width=900, plot_height=400, y_range=[-20000,20000])
# figGyro.line(source=sourceGx, x='x', y='y', line_width=2, alpha=.85, color='red')
# figGyro.line(source=sourceGy, x='x', y='y', line_width=2, alpha=.85, color='blue')
# figGyro.line(source=sourceGz, x='x', y='y', line_width=2, alpha=.85, color='green')

# handle = show(column(figAcc, figMag, figGyro))

Characteristic not found
Characteristic not found
